## **Libraries**

In [12]:
import os
import itertools
import numpy as np 
from joblib import Parallel, delayed

## **Environmental variable:** set the environmental variable to the folder where you downloaded the code

In [ ]:
os.environ["BASE_PATH"] = "Put your Path here"

## **Parameters**

In [4]:
train_rolling_length = 300 # rolling training window
test_rolling_length = 16 # rolling testing window
epochs = 2000 # number of epochs
batch_size = 32 # batch size
L_loss = ["listnet", "listnet_pairs", "listmle", "listmle_corrected"] # losses

ind_run = 1
L_ind_train = [i for i in range(20)]
L_short = ["bottom"]
L_pond = ["weighted"]
returns = np.load(f"{os.environ['BASE_PATH']}/raw_data/returns.npy", allow_pickle=True)
features = np.load(f"{os.environ['BASE_PATH']}/raw_data/features.npy", allow_pickle=True)
nb_features = features.shape[2]
nb_stocks = features.shape[1]
nb_long_max = nb_stocks//2
reb_fees = 0
freq = "weekly"
annualized_risk_free_rate = 0.02

## **Data**

In [6]:
from preprocessing import save_preprocessed_data

In [7]:
save_preprocessed_data(features, returns, train_rolling_length, test_rolling_length)

## **Training procedure**

In [8]:
from train import train_model_loss

In [10]:
param_combinations = list(
    itertools.product(
        L_loss,
        [L_ind_train],
        [train_rolling_length],
        [test_rolling_length],
        [epochs],
        [batch_size],
        [ind_run],
    )
)

In [ ]:
if __name__ == "__main__":
    # Number of parallel jobs to run
    num_jobs = (
        -1
    )  # Set to -1 to use all available CPU cores, or specify the desired number

    # Execute train function for each combination of parameters in parallel
    Parallel(n_jobs=num_jobs)(
        delayed(train_model_loss)(*param) for param in param_combinations
    )

## **Backtesting procedure**

In [5]:
from backtest import backtest

In [6]:
param_combinations = list(
    itertools.product(
        [L_ind_train],
        L_pond,
        L_short,
        [train_rolling_length],
        [test_rolling_length],
        [epochs],
        [batch_size],
        L_loss,
        [ind_run],
        [nb_features],
        [nb_stocks],
        [nb_long_max],
        [reb_fees],
    )
)

In [ ]:
for l in range(len(L_loss)):
    positions = np.zeros((nb_long_max, features.shape[0]-((features.shape[0]-train_rolling_length)%test_rolling_length)+1, nb_stocks))
    positions.setflags(write=True)
    backtest(*param_combinations[l], positions)

## **Out of sample financial metrics**

In [13]:
from src.ut_financial import get_resume_rank_metrics_test, get_resume_financial_metrics_test

In [14]:
df_rank_metrics = get_resume_rank_metrics_test(train_rolling_length, epochs, ind_run)
df_rank_metrics

,ListNet Classic,ListNet Fold,ListMLE Classic,ListMLE Weighted
ic,0.053,0.051,0.062,0.066
kendalltau,0.036,0.035,0.042,0.045
wkendalltau,0.044,0.045,0.054,0.059


In [15]:
df_financial_metrics = get_resume_financial_metrics_test(train_rolling_length, epochs, ind_run, freq, annualized_risk_free_rate)
df_financial_metrics

/Users/thibaut/Desktop/Templeton_final/Code/src/ut_financial.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_excess_returns_neg = df_excess_returns.applymap(lambda x: x if x < 0 else None)
/Users/thibaut/Desktop/Templeton_final/Code/src/ut_financial.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_excess_returns_neg = df_excess_returns.applymap(lambda x: x if x < 0 else None)
/Users/thibaut/Desktop/Templeton_final/Code/src/ut_financial.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_excess_returns_neg = df_excess_returns.applymap(lambda x: x if x < 0 else None)
/Users/thibaut/Desktop/Templeton_final/Code/src/ut_financial.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_excess_returns_neg = df_excess_returns.applymap(lambda x: x if x < 0 else None)


,ListNet Classic,ListNet Fold,ListMLE Classic,ListMLE Weighted
ex ret,0.16,0.17,0.20,0.25
vol,0.14,0.15,0.15,0.17
sharpe,1.16,1.17,1.36,1.49
MDD,-0.15,-0.14,-0.13,-0.13
sortino,1.49,1.58,1.78,1.99
VaR,-2.39,-2.74,-2.67,-2.64
